# Compiling data from the Bureau of Labor Statistics (BLS)


In [1]:
# Step 1: Import necessary functions and dataset

import pandas as pd
import numpy as np
import csv

data_url = 'https://data.bls.gov/cew/data/api/2019/3/industry/1026.csv'
d1 = pd.read_csv(data_url)

headers = ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'year', 'qtr', 'qtrly_estabs', 'month1_emplvl', 'month2_emplvl', 'month3_emplvl', 'total_qtrly_wages', 'avg_wkly_wage', 'lq_month1_emplvl', 'lq_month2_emplvl', 'lq_month3_emplvl', 'lq_total_qtrly_wages']

d2 = d1[headers]

In [2]:
# Only consider private, county-level stats

d3 = d2[d2['agglvl_code']==73]
d4 = d3[d3['own_code']==5]

d4.describe()

,own_code,industry_code,agglvl_code,year,qtr,qtrly_estabs,month1_emplvl,month2_emplvl,month3_emplvl,total_qtrly_wages,avg_wkly_wage,lq_month1_emplvl,lq_month2_emplvl,lq_month3_emplvl,lq_total_qtrly_wages
count,3264.0,3264.0,3264.0,3264.0,3264.0,3264.000000,3264.000000,3264.000000,3264.000000,3.264000e+03,3264.000000,3264.000000,3264.000000,3264.000000,3264.000000
mean,5.0,1026.0,73.0,2019.0,3.0,272.746324,5252.604167,5242.060662,5098.832414,3.224934e+07,323.469056,0.909409,0.905187,0.890313,0.953131
std,0.0,0.0,0.0,0.0,0.0,1034.352294,19728.178627,19687.332774,19479.534620,1.642614e+08,133.491218,0.576393,0.569938,0.550077,1.025436
min,5.0,1026.0,73.0,2019.0,3.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.0,1026.0,73.0,2019.0,3.0,23.000000,246.500000,244.000000,232.000000,8.568332e+05,260.000000,0.600000,0.600000,0.590000,0.490000
50%,5.0,1026.0,73.0,2019.0,3.0,59.000000,813.000000,809.000000,782.500000,3.248264e+06,302.000000,0.840000,0.830000,0.830000,0.730000
75%,5.0,1026.0,73.0,2019.0,3.0,172.000000,2940.250000,2934.750000,2802.250000,1.369625e+07,365.000000,1.070000,1.070000,1.060000,1.040000
max,5.0,1026.0,73.0,2019.0,3.0,39222.000000,549798.000000,548961.000000,547879.000000,5.019664e+09,1463.000000,7.090000,7.140000,7.340000,15.980000


In [3]:
# Combine month-level varibles into a quarter average

month_emplvl = ['month1_emplvl', 'month2_emplvl', 'month3_emplvl']
d5 = d4[month_emplvl]

qtr_emplvl = [np.mean(d5.iloc[row]) for row in range(len(d5))]
d4['qtr_emplvl'] = qtr_emplvl


lq_month_emplvl = ['lq_month1_emplvl', 'lq_month2_emplvl', 'lq_month3_emplvl']
d6 = d4[lq_month_emplvl]

lq_qtr_emplvl = [np.mean(d6.iloc[row]) for row in range(len(d6))]
d4['lq_qtr_emplvl'] = lq_qtr_emplvl

In [4]:
# Import new dataset to label FIPS with county name

import os
os.getcwd()
# os.chdir(r'C:\Users\ThomasWeinandy\OneDrive - BlueGranite, Inc\Blogs\covid19_economics') 

fips = pd.read_csv('area_titles.csv')
d7 = pd.merge(d4, fips, how='left', on='area_fips')

In [5]:
 # Round some digits, shed more variables, rename columns

d7['lq_qtr_emplvl'] = np.round(d7['lq_qtr_emplvl'], decimals=2)
d7['qtr_emplvl'] = d7['qtr_emplvl'].astype(int)

fewer_headers = ['area_fips', 'area_title', 'qtrly_estabs', 'qtr_emplvl', 'avg_wkly_wage', 'total_qtrly_wages', 'lq_qtr_emplvl']
d8 = d7[fewer_headers]

d8.rename(columns={'area_fips':'fips_id', 'area_title':'county_name', 'qtrly_estabs':'estab_count', 'qtr_emplvl':'emp_count', 'lq_qtr_emplvl':'percent_nat_avg' }, inplace=True)

d8.to_csv('county_leisure_emp.csv')